# Análisis de desempeño de modelos de XGBoost Individuales

En este código se realizó la construcción de un modelo por tipo de ataque. El documento está dividido en los siguientes apartados:

* **Cargado de base de datos:** Carga de la base de datos previamente transformada y en formato parquet para su uso en los modelos.
* **Modelo individual por tipo de ataque:** Diseño y construcción de un modelo específico para cada tipo de ataque, evaluando su desempeño individualmente.
* **Conclusiones:** Análisis de los resultados obtenidos de los diferentes modelos y planificación de los siguientes pasos a seguir.

# 1. Cargado de Base

Se utilizó la base de **CIC-DDoS2019**, que se carga a continuación junto con las demas librerías, que son las mismas del ejercicio de XGBoost:

In [ ]:
# Montado de drive para realizar el análisis
from google.colab import drive
drive.mount('/content/drive')
#Librerías Utilizadas
import pandas as pd # Manipulación de datos
pd.options.display.float_format = '{:.1f}'.format # Adecuar formato de salidas de tablas a 1 posición decimal
import seaborn as sns # Creación de gráficas
import matplotlib.pyplot as plt # Formato de gráficas
import numpy as np
# Herramientas de análisis de modelos
from sklearn.model_selection import train_test_split, GridSearchCV # Creación set entrenamiento - validación
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, recall_score, f1_score # Medición de precisión
from sklearn.preprocessing import LabelEncoder
# Librerías modelo XGbost
from xgboost import XGBClassifier # Modelo a entrenar
from xgboost import plot_importance # Gráfica para ver pesos de las variables

Mounted at /content/drive


Con estas librerías se realizó el cargue de la información, que corresponde a 431371 filas y 39 columnas y donde no hay valores nulos:

In [ ]:
df_CIC2019_corr = pd.read_parquet("/content/drive/MyDrive/Tesis - Machine Learning/Tesis - Machine Learning/E - Modelo predicción/Base/CICIDS2019_2.parquet")

In [ ]:
df_CIC2019_corr.head(5)

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Flow Bytes/s,Flow Packets/s,Fwd PSH Flags,Fwd Header Length,...,Fwd Seg Size Min,Label,Packet Length Mean,Fwd Packet Length Mean,Active Mean,Fwd IAT Mean,Idle Mean,Bwd IAT Mean,Bwd Packet Length Mean,Flow IAT Mean
0,17,48,4,0,1616.0,0.0,33666666.7,83333.3,0,80,...,20,DrDoS_NTP,411.2,404.0,0.0,16.0,0.0,0.0,0.0,16.0
1,17,557,112,0,49280.0,0.0,88473967.7,201077.2,0,0,...,0,DrDoS_NTP,440.0,440.0,0.0,5.0,0.0,0.0,0.0,5.0
2,6,323228,14,12,1736.0,526.0,6998.2,80.4,0,304,...,20,Benign,83.8,124.0,0.0,24863.7,0.0,15038.7,43.8,12929.1
3,17,46917,2,2,86.0,118.0,4348.1,85.3,0,40,...,20,Benign,49.4,43.0,0.0,3.0,0.0,2.0,59.0,15639.0
4,17,254,58,0,25520.0,0.0,100472440.9,228346.5,0,-58,...,-1,DrDoS_NTP,440.0,440.0,0.0,4.5,0.0,0.0,0.0,4.5


Adicional, se creó un dataframe con 2000 mil resgistros benignos para acompañar el entrenamiento de cada uno de los modelos.

In [ ]:
# Muestra de los benignos
df_temp=df_CIC2019_corr[df_CIC2019_corr["Label"]=="Benign"][:2000]

# Modelos de un tipo de ataque a la vez

Con esto en cuenta, se implementó el siguiente código, estructurado de la siguiente manera:

1. Se filtraron las etiquetas para conservar sólo los registros correspondientes a un tipo de ataque
2. Se añadió al dataframe una muestra de 2000 registros del caso benigno.
3. Se llevó a cabo el proceso de entrenamiento del modelo de XGBoost, que incluyó desde la división de la base de datos en entrenamiento y prueba hasta el entrenamiento y cálculo de métricas.
4. Por último, los resultados consolidados se almacenaron en un dataframe

In [ ]:
#Dataframe para guardado de resultados
df_total=pd.DataFrame()
#Código iterativo de modelos
for i in df_CIC2019_corr["Label"].unique():
  if i not in ["Label","Benign"]:
    print("*"*20)
    print(f"Predicción Modelo {i}")
    df_2=df_CIC2019_corr[df_CIC2019_corr["Label"].isin([i])]
    # Quedarnos con sólo una muestra de los benignos
    df_2=pd.concat([df_2,df_temp],axis=0)
    # Variables a entrenar
    X1 = df_2.drop(columns="Label")
    # Variable objetivo - sospechosos
    le = LabelEncoder()
    #Conversión de texto a numero
    encoded = le.fit_transform(df_2["Label"])
    y = encoded
    # División entrenamiento - test con ratio de 80 / 20
    X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.2, random_state=42)
    clf = XGBClassifier().fit(X_train, y_train)
    y_model = clf.predict(X_test)
    #Precisión
    acc = accuracy_score(y_test,y_model)
    print(f"Precisión: {acc}")
    #Recall
    rec = recall_score(y_test,y_model, average= "weighted") # Parametro weighted para calcular de acuerdo con cantidad
    print(f"Recall: {rec}")
    # F1 - Score
    f1 = f1_score(y_test,y_model, average= "weighted") # Parametro weighted para calcular de acuerdo con cantidad
    print(f"F1 score: {f1}")
    # Tomar importancia de modelo
    importance=clf.get_booster().get_score(importance_type='weight')
    # Convertir a DataFrame para mejor visualización
    importance_df = pd.DataFrame(importance.items(), columns=['Feature', 'Importance'])
    # Ordenar por importancia
    importance_df = importance_df.sort_values(by='Importance', ascending=False)
    print(importance_df.head(5))
    # Creación de dataframe resultados
    df_acum=pd.DataFrame()
    df_acum["Ataque"]=[i]
    df_acum["Precision"]=acc*100
    df_acum["Recall"]=rec*100
    df_acum["F1_score"]=f1*100
    df_acum["Main_Var"]=""
    df_acum.at[0,"Main_Var"]=list(importance_df["Feature"].head(5))
    df_total=pd.concat([df_total,df_acum])

********************
Predicción Modelo DrDoS_NTP
Precisión: 0.9997163005592932
Recall: 0.9997163005592932
F1 score: 0.9997164855205785
               Feature  Importance
19  Init Fwd Win Bytes       173.0
20  Init Bwd Win Bytes        85.0
11       Bwd Packets/s        79.0
22    Fwd Seg Size Min        62.0
1        Flow Duration        59.0
********************
Predicción Modelo TFTP
Precisión: 0.999900911613159
Recall: 0.999900911613159
F1 score: 0.999900911613159
                     Feature  Importance
20        Init Fwd Win Bytes       138.0
21        Init Bwd Win Bytes        66.0
26              Fwd IAT Mean        58.0
4   Fwd Packets Length Total        53.0
12             Bwd Packets/s        50.0
********************
Predicción Modelo Syn
Precisión: 0.9994160583941606
Recall: 0.9994160583941606
F1 score: 0.9994180600062533
               Feature  Importance
20  Init Fwd Win Bytes       112.0
21  Init Bwd Win Bytes        75.0
1        Flow Duration        60.0
13      ACK F

Los resultados presentados se resumieron en el siguiente dataframe, donde se agrupan las principales métricas de desempeño de los modelos y una columna adicional que muestra en orden de relevancia las principales variables:

In [ ]:
df_fin=df_total.reset_index().drop(columns="index")
df_fin

,Ataque,Precision,Recall,F1_score,Main_Var
0,DrDoS_NTP,100.0,100.0,100.0,"[Init Fwd Win Bytes, Init Bwd Win Bytes, Bwd P..."
1,TFTP,100.0,100.0,100.0,"[Init Fwd Win Bytes, Init Bwd Win Bytes, Fwd I..."
2,Syn,99.9,99.9,99.9,"[Init Fwd Win Bytes, Init Bwd Win Bytes, Flow ..."
3,UDP,100.0,100.0,100.0,"[Init Fwd Win Bytes, Avg Fwd Segment Size, Bwd..."
4,NetBIOS,99.5,99.5,99.5,"[Flow Bytes/s, Flow Duration, Fwd Packets Leng..."
5,MSSQL,100.0,100.0,100.0,"[Init Fwd Win Bytes, Protocol, Fwd Packets/s, ..."
6,DrDoS_DNS,99.7,99.7,99.7,"[Flow Bytes/s, Fwd Packets Length Total, Packe..."
7,LDAP,100.0,100.0,100.0,"[Avg Fwd Segment Size, Init Fwd Win Bytes, Fwd..."
8,DrDoS_SNMP,99.8,99.8,99.8,"[Avg Fwd Segment Size, Flow IAT Mean, Flow Byt..."
9,Portmap,99.6,99.6,99.6,"[Flow Bytes/s, Flow Duration, Bwd Packets/s, A..."


Puntualmente, se identifica que en todos los modelos se supera el 99% en todas las métricas, lo que sugiere que cada uno de los ataques tiene características que los distinguen de los casos benignos. Sin embargo, se observa que las 5 variables más importantes difieren entre modelos o están en un orden diferente, lo que indica que, en su estructura son diferentes.

# Comentarios finales

Si se necesita analizar un ataque específico en comparación con los casos benignos, se puede emplear un modelo más enfocado y especializado, que alcanza una precisión cercana al 100%. Esto está alineado con los hallazgos de otros estudios científicos que utilizan este dataset. No obstante, para los objetivos de este trabajo, no se ve pertinente este enfoque ya que se busca poder disminuir la cantidad de modelos para la detección de ciberataques.